# Network Training
Having implemented and tested all the components of the final networks in steps 1-3, we are now ready to train the network on a large dataset (ImageNet).

In [11]:
import gc
import datetime

import pandas as pd
import numpy as np

from copy import deepcopy
from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras import backend as K

import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from IPython.display import clear_output

from libs.pconv_model import PConvUnet
from libs.util import random_mask

%load_ext autoreload
%autoreload 2
plt.ioff()

# SETTINGS : NOISE = WHITE = 255
# MASK_NOISE = "/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0039_taken_640_broken/mask/mask.jpg"
MASK_NOISE = "/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/mask/mask.jpg"
# TEST_DIR_RUN = "/var/www2/trinh/Edited_Generative_Inpainting/training_data/training/GOPR0037_taken"
TEST_DIR_RUN = "/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input"

# MASKED_OUTPUT_DIR = "/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0037_taken/masked"
PRED_OUTPUT_DIR = "/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/output_pconv_160"

BATCH_SIZE = 1 # feed 1 by 1 image into network at test phase

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Creating train & test data generator

RUN FROM HERE

## Phase 3 - Generating samples

In [8]:
from keras.preprocessing import image

def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(512, 512))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]


    # convert black <-> white
    image_mask = cv2.imread(MASK_NOISE)
    image_mask = cv2.resize(image_mask,(512,512))
    image_mask[image_mask <=128] = 128
    image_mask[image_mask > 128] = 0
    image_mask[image_mask > 0] = 255
  
    
    # load conresponding mask
    # mask = np.stack([image_mask for _ in range(ori.shape[0])], axis=0) # ori shape : batchsize?
    mask = np.stack([image_mask], axis=0) # mask.shape: (1, 512, 512, 3)
    # print("mask.shape:", mask.shape)

    # random mask
    # mask = np.stack([random_mask(ori.shape[1], ori.shape[2]) for _ in range(ori.shape[0])], axis=0)
        
    # Apply masks to all image sample
    masked = deepcopy(img_tensor)
    masked[mask==0] = 1
    
    
    # show to test
    if show:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))
        axes[0].imshow(masked[0,:,:,:])
        axes[1].imshow(mask[0,:,:,:])
        axes[2].imshow(img_tensor[0,:,:,:])
        plt.show()
            
    return [masked, mask]


In [9]:
# Load weights from previous run
model = PConvUnet(weight_filepath='data/logs/')
model.load("data/logs/160_weights_2018-08-31-10-51-28.h5")
# model.load("data/logs/152_weights_2018-08-30-10-51-14.h5")


In [13]:
import os
import time




dir_files = os.listdir(TEST_DIR_RUN)
dir_files.sort()
count = 0

for image_item in dir_files:
    # count time
    start_time = time.time()
    
    base_file_name = os.path.basename(image_item) 
    print(TEST_DIR_RUN +"/" + base_file_name)
    img_path = TEST_DIR_RUN +"/" + base_file_name   
    # load a single image
    new_image = load_image(img_path, False)
    
    # check prediction
    pred = model.predict(new_image)
    print("pred.shape: ", pred.shape)
    print("--- Executed time: %s seconds ---" % (time.time() - start_time))
    
    # write images
    im = image.array_to_img(pred[0,:,:,:] * 1.)
    im.save( PRED_OUTPUT_DIR + "/" + base_file_name)
    
    
#     plt.figure(figsize=(4.5,4.5))
#     plt.imshow(im, interpolation='nearest')
#     plt.show()

    # plt.savefig("/home/ubuntu/trinh/Edited_Generative_Inpainting/training_data/testing/output/" + base_file_name)
    count +=1
#     if count> 5:
#         break
    

/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10635.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09998035430908203 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10636.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08978748321533203 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10637.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08881092071533203 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10638.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08223247528076172 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10639.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09139204025268555 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/r

pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08075499534606934 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10723.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09139204025268555 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10724.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.10817790031433105 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10725.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09675264358520508 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10726.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09859061241149902 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10727.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09959840774536133 seconds ---
/var

pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08185005187988281 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10768.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08789205551147461 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10769.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08074450492858887 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10770.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09446001052856445 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10771.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09259486198425293 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop10772.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09502363204956055 seconds ---
/var

/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11541.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09198641777038574 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11542.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08258271217346191 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11543.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.0930318832397461 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11544.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08417391777038574 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11545.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08366012573242188 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/ra

pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09315991401672363 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11587.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09597206115722656 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11588.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08631062507629395 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11589.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08261895179748535 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11590.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09586310386657715 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop11591.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09162306785583496 seconds ---
/var

...

pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08195948600769043 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop6970.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08097267150878906 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop6971.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.07733917236328125 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop6972.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08235692977905273 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop6973.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08154654502868652 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop6974.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08202147483825684 seconds ---
/var/www2

pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08363652229309082 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7016.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08225870132446289 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7017.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08480620384216309 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7018.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.07805228233337402 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7019.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08589744567871094 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7020.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08421802520751953 seconds ---
/var/www2

pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08490467071533203 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7833.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08068037033081055 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7834.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.0832371711730957 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7835.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08098816871643066 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7836.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.0804755687713623 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop7837.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08730340003967285 seconds ---
/var/www2/t

/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8131.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09152913093566895 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8132.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.07990074157714844 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8133.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09104228019714355 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8134.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08466577529907227 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8135.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08549022674560547 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindr

/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8317.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08833980560302734 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8318.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08073163032531738 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8319.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.08333659172058105 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8320.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.09149408340454102 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindrop8321.jpg
pred.shape:  (1, 512, 512, 3)
--- Executed time: 0.0849916934967041 seconds ---
/var/www2/trinh/Edited_Generative_Inpainting/test_dir/GOPR0268_line_broken_02/input/raindro